## list of models
- Full GRU
- Simple GRU
- CB-GRU
- CB-RNN-tied
- Dale-CB
- CB-RNN-tied-STP /
- Dale-CB-STP /
- Vanilla RNN

### Variants
With 24 neurons / with 48 neurons (let's do 48 first)

### Check Features
Input/ Ouput neurons

### Simple GRU

In [1]:
input_size = 8
sequence_length = 28*28//input_size
hidden_size = 24
num_layers = 1
num_classes = 10
batch_size = 40
num_epochs = 10
learning_rate = 0.01
stride_number = 4

In [2]:
'Data Preprocessing'
import torch
import math
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# get index of currently selected device
print(torch.cuda.current_device()) # returns 0 in my case

# get number of GPUs available
print(torch.cuda.device_count()) # returns 1 in my case

# get the name of the device
print(torch.cuda.get_device_name(0)) # good old Tesla K80

def snake_scan(img):
    'Converts a 32x32 image to a 32x96 array with snake-like row ordering'
    if len(img.shape) != 3:
        raise ValueError('Input image must be a 3D array')
    
    channels, rows, cols = img.shape
    snake = np.zeros((rows, cols * channels), dtype=img.dtype)
    for r in range(rows):
        row_data = img[:, r, :].flatten()  # Flattening each row into a 1D array of 96 elements
        if r % 2 == 1:
            row_data = row_data[::-1]  # Reversing the order for alternate rows
        snake[r] = row_data
    return snake

def stride(input_data, stride):
    'turn [batch_size, sequence_length, input_size] into [batch_size, sequence_length*input_size/stride, input_size]'
    batch_size, sequence_length, input_size = input_data.shape
    # flatten the input data to put sequence and input size together
    input_data = input_data.reshape(batch_size, -1)
    # append zeros to make sure the last pixel can be fed as the first pixel of the next sequence
    n = input_size - (sequence_length*input_size)%stride

    input_data = input_data.cpu()
    input_data = input_data.numpy()
    input_data = np.append(input_data, np.zeros((batch_size, n)), axis=1)
    input_data = torch.tensor(input_data)
    #print(input_data.shape)
    output_data = torch.zeros(batch_size, sequence_length*input_size//stride, input_size)
    for i in range(sequence_length*input_size//stride):
        # if stride = input size, then the output data is the same as input data
        #print(i)

        output_data[:,i,:] = input_data[:,i*stride:i*stride+input_size]
        #print(output_data[batch,i,:])

    return output_data

from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import Lambda

transform = transforms.Compose([
    ToTensor(),
    Lambda(lambda x: torch.tensor(snake_scan(x.numpy())))
])

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    download = True,
    transform = transform     
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    download = True,
    transform = transform
)



'Hyperparameters'
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=0),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=batch_size, 
                                          shuffle=False, 
                                          num_workers=0),
}
loaders
'''
for i, (images, labels) in enumerate(loaders['train']):
    images = images.reshape(-1, sequence_length, input_size).to(device)
    images = stride(images, stride_number).to(device)
    print(images.shape)
    print(labels.shape)
    print(len(loaders['train']))
    break
'''



Using device: cuda
0
1
NVIDIA GeForce RTX 4070 Laptop GPU


"\nfor i, (images, labels) in enumerate(loaders['train']):\n    images = images.reshape(-1, sequence_length, input_size).to(device)\n    images = stride(images, stride_number).to(device)\n    print(images.shape)\n    print(labels.shape)\n    print(len(loaders['train']))\n    break\n"

In [3]:
'Model Definition'
class simple_GRU_cell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(simple_GRU_cell, self).__init__()
        self.hidden_size = hidden_size
    
        # Rest gate r_t 
        self.W = torch.nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.P = torch.nn.Parameter(torch.rand(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.rand(self.hidden_size, 1))   

        # Update gate z_t
        # Wz is defined in the forward function
        self.W_z = torch.nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.P_z = torch.nn.Parameter(torch.rand(self.hidden_size, input_size))
        self.b_z = torch.nn.Parameter(torch.rand(self.hidden_size, 1))         

        # Firing rate, Scaling factor and time step initialization
        self.r_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # Nonlinear functions
        self.Sigmoid = nn.Sigmoid()
        self.Tanh = nn.Tanh()
        for name, param in self.named_parameters():
            nn.init.uniform_(param, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))

    def forward(self, x):        
        if self.r_t.dim() == 3:           
            self.r_t = self.r_t[0]
        self.r_t = torch.transpose(self.r_t, 0, 1)
        self.z_t = self.Sigmoid(torch.matmul(self.W_z, self.r_t) + torch.matmul(self.P_z, x) + self.b_z)

        # input mask
        # we want this to be orthogonal to the E/I split, so zero out half of excitatory neurons and half of inhibitory neurons
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//2:,:] = 0
        P = self.P * input_mask

        self.r_t = (1 - self.z_t) * self.r_t + self.z_t * self.Sigmoid(torch.matmul(self.W, self.r_t) + torch.matmul(P, x) + self.b_v)
        self.r_t = torch.transpose(self.r_t, 0, 1)                

class simple_GRU_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(simple_GRU_batch, self).__init__()
        self.rnncell = simple_GRU_cell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.r_t             
            
class simple_GRU(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(simple_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = simple_GRU_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.r_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Passing in the input and hidden state into the model and  obtaining outputs
        out = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        # output mask
        output_mask = torch.ones_like(out)
        output_mask[:,0:self.hidden_size//2] = 0
        out = out * output_mask
        
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass
model = simple_GRU(input_size, hidden_size, num_layers, num_classes).to(device)

In [4]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


simple_GRU(
  (lstm): simple_GRU_batch(
    (rnncell): simple_GRU_cell(
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
  )
  (fc): Linear(in_features=24, out_features=10, bias=True)
)
Epoch [1/10], Step [750/1500], Training Accuracy: 53.10
Epoch [1/10], Step [1500/1500], Training Accuracy: 61.40
Epoch [2/10], Step [750/1500], Training Accuracy: 75.40
Epoch [2/10], Step [1500/1500], Training Accuracy: 80.30
Epoch [3/10], Step [750/1500], Training Accuracy: 81.60
Epoch [3/10], Step [1500/1500], Training Accuracy: 82.50
Epoch [4/10], Step [750/1500], Training Accuracy: 83.30
Epoch [4/10], Step [1500/1500], Training Accuracy: 85.00
Epoch [5/10], Step [750/1500], Training Accuracy: 86.00
Epoch [5/10], Step [1500/1500], Training Accuracy: 87.40
Epoch [6/10], Step [750/1500], Training Accuracy: 87.80
Epoch [6/10], Step [1500/1500], Training Accuracy: 86.50
Epoch [7/10], Step [750/1500], Training Accuracy: 87.80
No improvement in validation accuracy for 2 epochs. Stopping training.


In [5]:
'Testing Accuracy'
# Test the model
model.eval()
labelslist = []
with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:

        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()

        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))


Accuracy of the model:90.03%


In [6]:
'Save Model'
# Retrieve weights
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
W_z = model.lstm.rnncell.W_z.detach().cpu().numpy()
P_z = model.lstm.rnncell.P_z.detach().cpu().numpy()
b_z = model.lstm.rnncell.b_z.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()


import pickle
with open('weights24/02_simple_GRU_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, W_z, P_z, b_z, b_v], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()

In [7]:
# save the trained model
torch.save(model.state_dict(), 'weights24/02_simple_GRU_48.pth')

### CB-GRU

In [8]:
'Model Definition'

class CB_GRU_cell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CB_GRU_cell, self).__init__()
        self.hidden_size = hidden_size
    
        # Rest gate r_t 
        self.W = torch.nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.P = torch.nn.Parameter(torch.rand(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.rand(self.hidden_size, 1))   

        # Update gate z_t
        # K is always positive            
        self.b_z = torch.nn.Parameter(torch.rand(self.hidden_size, 1))     
        self.K = torch.nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.P_z = torch.nn.Parameter(torch.rand(self.hidden_size, input_size))

        # Firing rate, Scaling factor and time step initialization
        self.v_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # dt is a constant
        self.dt = nn.Parameter(torch.tensor(0.1), requires_grad = False)

        # Nonlinear functions
        self.Sigmoid = nn.Sigmoid()
        self.Tanh = nn.Tanh()
        for name, param in self.named_parameters():
            nn.init.uniform_(param, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))
    @property
    def r_t(self):
        return self.Sigmoid(self.v_t)

    def forward(self, x):        
        if self.v_t.dim() == 3:           
            self.v_t = self.v_t[0]
        self.v_t = torch.transpose(self.v_t, 0, 1)
        # No sign constraint on K and W

        # input mask
        # we want this to be orthogonal to the E/I split, so zero out half of excitatory neurons and half of inhibitory neurons
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//2:,:] = 0
        P = self.P * input_mask
        
        self.z_t = torch.zeros(self.hidden_size, 1)
        self.z_t = self.dt * self.Sigmoid(torch.matmul(self.K , self.r_t) + torch.matmul(self.P_z, x) + self.b_z)
        self.v_t = (1 - self.z_t) * self.v_t + self.dt * (torch.matmul(self.W, self.r_t) + torch.matmul(P, x) + self.b_v)
        self.v_t = torch.transpose(self.v_t, 0, 1)                

class CB_GRU_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(CB_GRU_batch, self).__init__()
        self.rnncell = CB_GRU_cell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.v_t             
            
class CB_GRU(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CB_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = CB_GRU_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.v_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Passing in the input and hidden state into the model and  obtaining outputs
        out = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        #Reshaping the outputs such that it can be fit into the fully connected layer
        output_mask = torch.ones_like(out)
        output_mask[:,0:self.hidden_size//2] = 0
        out = out * output_mask
        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass
model = CB_GRU(input_size, hidden_size, num_layers, num_classes).to(device)

In [9]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


CB_GRU(
  (lstm): CB_GRU_batch(
    (rnncell): CB_GRU_cell(
      (Sigmoid): Sigmoid()
      (Tanh): Tanh()
    )
  )
  (fc): Linear(in_features=24, out_features=10, bias=True)
)
Epoch [1/10], Step [750/1500], Training Accuracy: 24.60


In [ ]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


In [ ]:
'Testing Accuracy'
# Test the model
model.eval()
labelslist = []
with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:

        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()

        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))

# Retrieve weights
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()


Accuracy of the model:83.39%


In [ ]:
'Save Model'
# Retrieve weights
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
K = model.lstm.rnncell.K.detach().cpu().numpy()
P_z = model.lstm.rnncell.P_z.detach().cpu().numpy()
b_z = model.lstm.rnncell.b_z.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()


import pickle
with open('weights24/03_CB_GRU_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, K, P_z, b_z, b_v], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()

In [ ]:
# save the trained model
torch.save(model.state_dict(), 'weights24/03_CB_GRU_48.pth')

### CB-RNN-tied


In [ ]:
class CB_RNN_tiedcell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CB_RNN_tiedcell, self).__init__()
        self.hidden_size = hidden_size
    
        ### Parameters ###
        # voltage gate v_t 
        self.W = torch.nn.Parameter(torch.empty(self.hidden_size, self.hidden_size))
        self.P = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.zeros(self.hidden_size, 1))   

        # Update gate z_t
        # K and P_z become tied          
        self.b_z = torch.nn.Parameter(torch.empty(self.hidden_size, 1))   
        # initialise e as a random float between 0 and 1
        self.e = torch.nn.Parameter(torch.rand(1))
        self.e_p = torch.nn.Parameter(torch.rand(1))

        # Voltage rate
        self.v_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # dt is a constant
        self.dt = nn.Parameter(torch.tensor(0.1), requires_grad = False)

        ### Nonlinear functions ###
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        ### Initialisation ###
        glorot_init = lambda w: nn.init.uniform_(w, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))
        positive_glorot_init = lambda w: nn.init.uniform_(w, a=0, b=(1/math.sqrt(hidden_size)))

        # initialise matrices
        for w in self.W, self.P:
            glorot_init(w)
        # init b_z to be log 1/99
        nn.init.constant_(self.b_z, torch.log(torch.tensor(1/99)))

        

    @property
    def r_t(self):
        return self.sigmoid(self.v_t)

    def forward(self, x):        
        if self.v_t.dim() == 3:           
            self.v_t = self.v_t[0]
        self.v_t = torch.transpose(self.v_t, 0, 1)

        ### Constraints###
        e = self.softplus(self.e)
        e_p = self.softplus(self.e_p)
        K = e * self.softplus(self.W)
        P_z = e_p * self.softplus(self.P)


        ### Update Equations ###
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//2:,:] = 0
        P = self.P * input_mask

        self.z_t = torch.zeros(self.hidden_size, 1)
        self.z_t = self.dt * self.sigmoid(torch.matmul(K , self.r_t) + torch.matmul(P_z, x) + self.b_z)
        self.v_t = (1 - self.z_t) * self.v_t + self.dt * (torch.matmul(self.W, self.r_t) + torch.matmul(P, x) + self.b_v)
        self.v_t = torch.transpose(self.v_t, 0, 1)                

class CB_RNN_tied_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(CB_RNN_tied_batch, self).__init__()
        self.rnncell = CB_RNN_tiedcell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.v_t             
            
class CB_RNN_tied(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CB_RNN_tied, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = CB_RNN_tied_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.v_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # Passing in the input and hidden state into the model and  obtaining outputs
        out = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        # output mask
        output_mask = torch.ones_like(out)
        output_mask[:,0:self.hidden_size//2] = 0
        out = out * output_mask

        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass

model = CB_RNN_tied(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


CB_RNN_tied(
  (lstm): CB_RNN_tied_batch(
    (rnncell): CB_RNN_tiedcell(
      (sigmoid): Sigmoid()
      (softplus): Softplus(beta=1, threshold=20)
      (relu): ReLU()
    )
  )
  (fc): Linear(in_features=48, out_features=10, bias=True)
)
Epoch [1/10], Step [750/1500], Training Accuracy: 53.90
Epoch [1/10], Step [1500/1500], Training Accuracy: 57.20
Epoch [2/10], Step [750/1500], Training Accuracy: 65.70
Epoch [2/10], Step [1500/1500], Training Accuracy: 70.40
Epoch [3/10], Step [750/1500], Training Accuracy: 74.00
Epoch [3/10], Step [1500/1500], Training Accuracy: 77.40
Epoch [4/10], Step [750/1500], Training Accuracy: 77.20
Epoch [4/10], Step [1500/1500], Training Accuracy: 80.10
Epoch [5/10], Step [750/1500], Training Accuracy: 78.00
Epoch [5/10], Step [1500/1500], Training Accuracy: 80.70
Epoch [6/10], Step [750/1500], Training Accuracy: 85.10
Epoch [6/10], Step [1500/1500], Training Accuracy: 84.50
Epoch [7/10], Step [750/1500], Training Accuracy: 82.30
No improvement in valida

In [ ]:
'Testing Accuracy'
# Test the model
model.eval()
labelslist = []
with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:

        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()

        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))



Accuracy of the model:83.95%


In [ ]:
'Save Model'
# Retrieve weights
e = model.lstm.rnncell.e
e_p = model.lstm.rnncell.e_p
K = e * nn.Softplus()(model.lstm.rnncell.W)
P_z = e_p * nn.Softplus()(model.lstm.rnncell.P)
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
e = model.lstm.rnncell.e.detach().cpu().numpy()
e_p = model.lstm.rnncell.e_p.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()


import pickle
with open('weights24/04_CB-RNN-tied_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, K, P_z, b_v, e, e_p], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()

In [ ]:
# save the trained model
torch.save(model.state_dict(), 'weights24/04_CB-RNN-tied_48.pth')

### Dale-CB

In [ ]:
class Dale_CBcell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Dale_CBcell, self).__init__()
        self.hidden_size = hidden_size
    
        ### Parameters ###
        # voltage gate v_t 
        self.P = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.zeros(self.hidden_size, 1))   

        # Update gate z_t
        # K and W are unbounded free parameters   
        # C represents  current based portion of connectivity       
        self.K = torch.nn.Parameter(self.init_dale(self.hidden_size, self.hidden_size))
        self.C = torch.nn.Parameter(self.init_dale(self.hidden_size, self.hidden_size))
        self.P_z = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))
        self.b_z = torch.nn.Parameter(torch.empty(self.hidden_size, 1))   
        # Potentials are initialised with right signs
        self.e_e = torch.nn.Parameter(torch.rand(1))
        self.e_i = torch.nn.Parameter(-torch.rand(1))

        # Firing rate, Scaling factor and time step initialization
        self.v_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # dt is a constant
        self.dt = nn.Parameter(torch.tensor(0.1), requires_grad = False)

        ### Nonlinear functions ###
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        ### Initialisation ###
        glorot_init = lambda w: nn.init.uniform_(w, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))
        positive_glorot_init = lambda w: nn.init.uniform_(w, a=0, b=(1/math.sqrt(hidden_size)))

        # initialise matrices
        # P and P_z are unconstrained
        for w in self.P_z, self.P:
            glorot_init(w)
        for w in self.K, self.C:
            positive_glorot_init(w)
        # init b_z to be log 1/99
        nn.init.constant_(self.b_z, torch.log(torch.tensor(1/99)))
        self.v_t_history = []
        self.z_t_history = []

    def init_dale(self, rows, cols):
        # Dale's law with equal excitatory and inhibitory neurons
        exci = torch.empty((rows, cols//2)).exponential_(1.0)
        inhi = -torch.empty((rows, cols//2)).exponential_(1.0)
        weights = torch.cat((exci, inhi), dim=1)
        weights = self.adjust_spectral(weights)
        return weights

    def adjust_spectral(self, weights, desired_radius=1.5):
        #values, _ = torch.linalg.eig(weights @ weights.T)
        values = torch.linalg.svdvals(weights)
        radius = values.abs().max()
        return weights * (desired_radius / radius)
        

    @property
    def r_t(self):
        return self.sigmoid(self.v_t)

    def forward(self, x):        
        if self.v_t.dim() == 3:           
            self.v_t = self.v_t[0]
        self.v_t = torch.transpose(self.v_t, 0, 1)

        ### Constraints###
        K = self.softplus(self.K)
        C = self.softplus(self.C)
        # W is constructed using e*(K+C)
        W_E = self.e_e * (K[:, :self.hidden_size//2] + C[:, :self.hidden_size//2])
        W_I = self.e_i * (K[:, self.hidden_size//2:] + C[:, self.hidden_size//2:])
        # print to see which device the tensor is on
        # If sign of W do not obey Dale's law, then these terms to be 0
        W_E = self.relu(W_E)
        W_I = -self.relu(-W_I)
        W = torch.cat((W_E, W_I), 1)
        self.W = W

        ### Update Equations ###
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//4:self.hidden_size//2,:] = 0
        input_mask[3*self.hidden_size//4:,:] = 0
        P = self.P * input_mask

        self.z_t = torch.zeros(self.hidden_size, 1)
        self.z_t = self.dt * self.sigmoid(torch.matmul(K , self.r_t) + torch.matmul(self.P_z, x) + self.b_z)
        self.v_t = (1 - self.z_t) * self.v_t + self.dt * (torch.matmul(W, self.r_t) + torch.matmul(P, x) + self.b_v)
        self.v_t = torch.transpose(self.v_t, 0, 1)      
        excitatory = self.v_t[:, :self.hidden_size//2]
        self.excitatory = torch.cat((excitatory, torch.zeros_like(excitatory)), 1)

        self.v_t_history.append(self.v_t)
        self.z_t_history.append(self.z_t)    

class Dale_CB_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(Dale_CB_batch, self).__init__()
        self.rnncell = Dale_CBcell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.excitatory            
            
class Dale_CB(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Dale_CB, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = Dale_CB_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.v_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Passing in the input and hidden state into the model and  obtaining outputs
        out = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        #Reshaping the outputs such that it can be fit into the fully connected layer
        # output mask
        output_mask = torch.ones_like(out)
        output_mask[:,:self.hidden_size//4] = 0
        output_mask[:,3*self.hidden_size//4:] = 0        
        out = out * output_mask
        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass

model = Dale_CB(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


Dale_CB(
  (lstm): Dale_CB_batch(
    (rnncell): Dale_CBcell(
      (sigmoid): Sigmoid()
      (softplus): Softplus(beta=1, threshold=20)
      (relu): ReLU()
    )
  )
  (fc): Linear(in_features=48, out_features=10, bias=True)
)
Epoch [1/10], Step [750/1500], Training Accuracy: 42.30
Epoch [1/10], Step [1500/1500], Training Accuracy: 59.30
Epoch [2/10], Step [750/1500], Training Accuracy: 65.40
Epoch [2/10], Step [1500/1500], Training Accuracy: 70.60
Epoch [3/10], Step [750/1500], Training Accuracy: 74.00
Epoch [3/10], Step [1500/1500], Training Accuracy: 75.00
Epoch [4/10], Step [750/1500], Training Accuracy: 71.40
Epoch [4/10], Step [1500/1500], Training Accuracy: 77.50
Epoch [5/10], Step [750/1500], Training Accuracy: 74.60
Epoch [5/10], Step [1500/1500], Training Accuracy: 79.90
Epoch [6/10], Step [750/1500], Training Accuracy: 81.00
Epoch [6/10], Step [1500/1500], Training Accuracy: 81.90
Epoch [7/10], Step [750/1500], Training Accuracy: 82.60
Epoch [7/10], Step [1500/1500], Trai

In [ ]:
'Testing Accuracy'
# Test the model
model.eval()
v_t_history = []
z_t_history = []
labelslist = []

with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        model.lstm.rnncell.v_t_history = []
        model.lstm.rnncell.z_t_history = []
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()

        v_t_history.append(model.lstm.rnncell.v_t_history)
        z_t_history.append(model.lstm.rnncell.z_t_history)
        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))

Accuracy of the model:82.48%


In [ ]:
'Save Model'
# Retrieve weights
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
K = model.lstm.rnncell.K.detach().cpu().numpy()
C = model.lstm.rnncell.C.detach().cpu().numpy()
P_z = model.lstm.rnncell.P_z.detach().cpu().numpy()
b_z = model.lstm.rnncell.b_z.detach().cpu().numpy()
e_e = model.lstm.rnncell.e_e.detach().cpu().numpy()
e_i = model.lstm.rnncell.e_i.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()


import pickle
with open('weights24/05_Dale-CB_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, K, C, P_z, b_z, e_e, e_i, b_v], f)
    pickle.dump([v_t_history, z_t_history, labelslist], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()

In [ ]:
# save the trained model
torch.save(model.state_dict(), 'weights24/05_Dale-CB_48.pth')

### CB-RNN-tied-STP


In [ ]:
'Model Definition'

class CB_RNN_tiedcell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CB_RNN_tiedcell, self).__init__()
        self.hidden_size = hidden_size
    
        ### Parameters ###
        # voltage gate v_t 
        self.W = torch.nn.Parameter(torch.empty(self.hidden_size, self.hidden_size))
        self.P = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.zeros(self.hidden_size, 1))   

        # Update gate z_t
        # K and P_z become tied          
        self.b_z = torch.nn.Parameter(torch.empty(self.hidden_size, 1))   
        # initialise e as a random float between 0 and 1
        self.e = torch.nn.Parameter(torch.rand(1))
        self.e_p = torch.nn.Parameter(torch.rand(1))

        # Voltage rate
        self.v_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # dt is a constant
        self.dt = nn.Parameter(torch.tensor(1.0), requires_grad = False)
        self.z_high = 0.2

        ### Nonlinear functions ###
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        ### Initialisation ###
        glorot_init = lambda w: nn.init.uniform_(w, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))
        positive_glorot_init = lambda w: nn.init.uniform_(w, a=0, b=(1/math.sqrt(hidden_size)))

        # initialise matrices
        for w in self.W, self.P:
            glorot_init(w)
        # init b_z to be log 1/99
        nn.init.constant_(self.b_z, torch.log(torch.tensor(1/99)))

        ### STP Model ###
        self.delta_t = 1
        self.z_min = 0.001
        self.z_max = 0.1

        # Short term Depression parameters  
        self.c_x = torch.nn.Parameter(torch.rand(self.hidden_size, 1))

        # Short term Facilitation parameters
        self.c_u = torch.nn.Parameter(torch.rand(self.hidden_size, 1))
        self.c_U = torch.nn.Parameter(torch.rand(self.hidden_size, 1))
        
        # State initialisations
        self.X = torch.ones(self.hidden_size, 1, dtype=torch.float32).to(device)
        self.U = torch.full((self.hidden_size, 1), 0.9, dtype=torch.float32).to(device)
        self.Ucap = 0.9 * self.sigmoid(self.c_U)
        self.Ucapclone = self.Ucap.clone().detach() 

        self.X_history = []
        self.U_history = []
        self.v_t_history = []
        self.z_t_history = []

        

    @property
    def r_t(self):
        return self.sigmoid(self.v_t)

    def forward(self, x):        
        if self.v_t.dim() == 3:           
            self.v_t = self.v_t[0]
        self.v_t = torch.transpose(self.v_t, 0, 1)

        ### Constraints###
        e = self.softplus(self.e)
        e_p = self.softplus(self.e_p)
        K = e * self.softplus(self.W)
        P_z = e_p * self.softplus(self.P)

        ### STP model ###
        x = torch.transpose(x, 0, 1)
        
        # Short term Depression 
        self.z_x = self.z_min + (self.z_max - self.z_min) * self.sigmoid(self.c_x)
        self.X = self.z_x + torch.mul((1 - self.z_x), self.X) - self.delta_t * self.U * self.X * self.r_t

        # Short term Facilitation 
        self.z_u = self.z_min + (self.z_max - self.z_min) * self.sigmoid(self.c_u)    
        self.Ucap = 0.9 * self.sigmoid(self.c_U)
        self.U = self.Ucap * self.z_u + torch.mul((1 - self.z_u), self.U) + self.delta_t * self.Ucap * (1 - self.U) * self.r_t
        self.Ucapclone = self.Ucap.clone().detach()
        self.U = torch.clamp(self.U, min=self.Ucapclone.repeat(1, x.size(0)).to(device), max=torch.ones_like(self.Ucapclone.repeat(1, x.size(0)).to(device)))
        x = torch.transpose(x, 0, 1)

        ### Update Equations ###
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//2:,:] = 0
        P = self.P * input_mask
        self.z_t = self.dt * self.z_high* self.sigmoid(torch.matmul(K , self.r_t) + torch.matmul(P_z, x) + self.b_z)
        # mask p with second half of the neuron not receiving input

        self.v_t = (1 - self.z_t) * self.v_t + self.dt * (torch.matmul(self.W, self.U*self.X*self.r_t) + torch.matmul(P, x) + self.b_v)
        self.v_t = torch.transpose(self.v_t, 0, 1)      

        self.X_history.append(self.X.clone().detach())
        self.U_history.append(self.U.clone().detach())
        self.v_t_history.append(self.v_t.clone().detach())
        self.z_t_history.append(self.z_t.clone().detach())       

class CB_RNN_tied_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(CB_RNN_tied_batch, self).__init__()
        self.rnncell = CB_RNN_tiedcell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.v_t             
            
class CB_RNN_tied(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CB_RNN_tied, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = CB_RNN_tied_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.X = torch.ones(self.hidden_size, x.size(0), dtype=torch.float32).to(device)
        self.lstm.rnncell.U = (self.lstm.rnncell.Ucapclone.repeat(1, x.size(0))).to(device)
        self.lstm.rnncell.v_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # Passing in the input and hidden state into the model and  obtaining outputs
        # mask only the second half giving output
        out = self.lstm(x)  # out: tensor of shape (batch_size, hidden_size)
        output_mask = torch.ones_like(out)
        output_mask[:,0:self.hidden_size//2] = 0
        out = out * output_mask
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass

model = CB_RNN_tied(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


CB_RNN_tied(
  (lstm): CB_RNN_tied_batch(
    (rnncell): CB_RNN_tiedcell(
      (sigmoid): Sigmoid()
      (softplus): Softplus(beta=1, threshold=20)
      (relu): ReLU()
    )
  )
  (fc): Linear(in_features=48, out_features=10, bias=True)
)


In [ ]:
'Testing Accuracy'
# Test the model
model.eval()
X_history = []
U_history = []
v_t_history = []
z_t_history = []
labelslist = []

with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        model.lstm.rnncell.X_history = []
        model.lstm.rnncell.U_history = []
        model.lstm.rnncell.v_t_history = []
        model.lstm.rnncell.z_t_history = []
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()
        X_history.append(model.lstm.rnncell.X_history)
        U_history.append(model.lstm.rnncell.U_history)
        v_t_history.append(model.lstm.rnncell.v_t_history)
        z_t_history.append(model.lstm.rnncell.z_t_history)
        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))


In [ ]:
# Save Model
# Retrieve weights
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
Ucap = model.lstm.rnncell.Ucap.detach().cpu().numpy()
c_U = model.lstm.rnncell.c_U.detach().cpu().numpy()
c_u = model.lstm.rnncell.c_u.detach().cpu().numpy()
c_x = model.lstm.rnncell.c_x.detach().cpu().numpy()
e = model.lstm.rnncell.e.detach().cpu().numpy()
e_p = model.lstm.rnncell.e_p.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()
K = e * model.softplus(model.W)
P_z = e_p * model.softplus(model.P)

import pickle
with open('weights24/06_CB-RNN-tied-STP_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, K, P_z, b_v, Ucap, c_U, c_u, c_x, e, e_p], f)
    pickle.dump([X_history, U_history, v_t_history, z_t_history, labelslist], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()


In [ ]:
# save the trained model
torch.save(model.state_dict(), 'weights24/06_CB-RNN-tied-STP_48.pth')

### Dale-CB-STP
Accuracy of the model:55.56% (doubled neurons)

In [ ]:
class Dale_CB_STPcell(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Dale_CB_STPcell, self).__init__()
        self.hidden_size = hidden_size
    
        ### Parameters ###
        # voltage gate v_t 
        self.P = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))           
        self.b_v = torch.nn.Parameter(torch.zeros(self.hidden_size, 1))   

        # Update gate z_t
        # K and W are unbounded free parameters   
        # C represents  current based portion of connectivity       
        while True:
            self.K = torch.nn.Parameter(self.init_dale(self.hidden_size, self.hidden_size))
            self.C = torch.nn.Parameter(self.init_dale(self.hidden_size, self.hidden_size))
            nse_K = self.NSE(self.K)
            nse_C = self.NSE(self.C)
            if nse_K > 0.87 and nse_C > 0.87:
                break
        self.P_z = torch.nn.Parameter(torch.empty(self.hidden_size, input_size))
        self.b_z = torch.nn.Parameter(torch.empty(self.hidden_size, 1))   
        # Potentials are initialised with right signs
        self.e_e = torch.nn.Parameter(torch.rand(1))
        self.e_i = torch.nn.Parameter(-torch.rand(1))

        # Firing rate, Scaling factor and time step initialization
        self.v_t = torch.zeros(1, self.hidden_size, dtype=torch.float32)

        # dt is a constant
        self.dt = nn.Parameter(torch.tensor(1.0), requires_grad = False)
        self.z_high = nn.Parameter(torch.repeat_interleave(torch.tensor(0.2), self.hidden_size).reshape(self.hidden_size,1), requires_grad = False)
        self.z_low = torch.nn.Parameter(torch.zeros(self.hidden_size, 1, dtype=torch.float32), requires_grad = False)
        self.z_low[self.hidden_size//2:,:] = 0.1

        ### Nonlinear functions ###
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
        self.relu = nn.ReLU()

        ### Initialisation ###
        glorot_init = lambda w: nn.init.uniform_(w, a=-(1/math.sqrt(hidden_size)), b=(1/math.sqrt(hidden_size)))
        positive_glorot_init = lambda w: nn.init.uniform_(w, a=0, b=(1/math.sqrt(hidden_size)))

        # initialise matrices
        # P and P_z are unconstrained
        for w in self.P_z, self.P:
            glorot_init(w)
        for w in self.K, self.C:
            positive_glorot_init(w)
        # init b_z to be log 1/99
        nn.init.constant_(self.b_z, torch.log(torch.tensor(1/99)))

        ### STP Model ###
        self.delta_t = 1
        self.z_min = 0.001
        self.z_max = 0.1

        # Short term Depression parameters  
        self.c_x = torch.nn.Parameter(torch.rand(self.hidden_size, 1))

        # Short term Facilitation parameters
        self.c_u = torch.nn.Parameter(torch.rand(self.hidden_size, 1))
        self.c_U = torch.nn.Parameter(torch.rand(self.hidden_size, 1))
        
        # State initialisations
        self.X = torch.ones(self.hidden_size, 1, dtype=torch.float32).to(device)
        self.U = torch.full((self.hidden_size, 1), 0.9, dtype=torch.float32).to(device)
        self.Ucap = 0.9 * self.sigmoid(self.c_U)
        self.Ucapclone = self.Ucap.clone().detach() 

        self.X_history = []
        self.U_history = []
        self.v_t_history = []
        self.z_t_history = []

    def init_dale(self, rows, cols):
        # Dale's law with equal excitatory and inhibitory neurons
        exci = torch.empty((rows, cols//2)).exponential_(1.0)
        inhi = -torch.empty((rows, cols//2)).exponential_(1.0)
        weights = torch.cat((exci, inhi), dim=1)
        weights = self.adjust_spectral(weights)
        return weights

    def adjust_spectral(self, weights, desired_radius=1.5):
        values= torch.linalg.svdvals(weights)
        radius = values.abs().max()
        return weights * (desired_radius / radius)
    
    def NSE(self, weights):
        values = torch.linalg.svdvals(weights)
        normalised_v = values/sum(values)
        H = -1/torch.log(torch.tensor(self.hidden_size)) * torch.sum(normalised_v * torch.log(normalised_v))
        print(H)
        return H

    @property
    def r_t(self):
        return self.sigmoid(self.v_t)

    def forward(self, x):        
        if self.v_t.dim() == 3:           
            self.v_t = self.v_t[0]
        self.v_t = torch.transpose(self.v_t, 0, 1)

        ### Constraints###
        K = self.softplus(self.K)
        C = self.softplus(self.C)
        # W is constructed using e*(K+C)
        # first half of neurons are excitatory and second half are inhibitory
        W_E = self.e_e * (K[:, :self.hidden_size//2] + C[:, :self.hidden_size//2])
        W_I = self.e_i * (K[:, self.hidden_size//2:] + C[:, self.hidden_size//2:])
        # print to see which device the tensor is on
        # If sign of W do not obey Dale's law, then these terms to be 0
        W_E = self.relu(W_E)
        W_I = -self.relu(-W_I)
        W = torch.cat((W_E, W_I), 1)
        self.W = W

        ### STP model ###
        x = torch.transpose(x, 0, 1)
        
        # Short term Depression 
        self.z_x = self.z_min + (self.z_max - self.z_min) * self.sigmoid(self.c_x)
        self.X = self.z_x + torch.mul((1 - self.z_x), self.X) - self.delta_t * self.U * self.X * self.r_t

        # Short term Facilitation 
        self.z_u = self.z_min + (self.z_max - self.z_min) * self.sigmoid(self.c_u)    
        self.Ucap = 0.9 * self.sigmoid(self.c_U)
        self.U = self.Ucap * self.z_u + torch.mul((1 - self.z_u), self.U) + self.delta_t * self.Ucap * (1 - self.U) * self.r_t
        self.Ucapclone = self.Ucap.clone().detach()
        self.U = torch.clamp(self.U, min=self.Ucapclone.repeat(1, x.size(0)).to(device), max=torch.ones_like(self.Ucapclone.repeat(1, x.size(0)).to(device)))
        x = torch.transpose(x, 0, 1)


        ### Update Equations ###
        self.z_t = torch.zeros(self.hidden_size, 1)
        self.z_t = self.dt*self.z_low + self.dt * (self.z_high - self.z_low)*self.sigmoid(torch.matmul(K , self.r_t) + torch.matmul(self.P_z, x) + self.b_z)
        
        # input mask
        # we want this to be orthogonal to the E/I split, so zero out half of excitatory neurons and half of inhibitory neurons
        input_mask = torch.ones_like(self.P)
        input_mask[self.hidden_size//4:self.hidden_size//2,:] = 0
        input_mask[3*self.hidden_size//4:,:] = 0
        P = self.P * input_mask

        self.v_t = (1 - self.z_t) * self.v_t + self.dt * (torch.matmul(W, self.U*self.X*self.r_t) + torch.matmul(P, x) + self.b_v)
        self.v_t = torch.transpose(self.v_t, 0, 1)      
        excitatory = self.v_t[:, :self.hidden_size//2]
        self.excitatory = torch.cat((excitatory, torch.zeros_like(excitatory)), 1)    

        self.X_history.append(self.X.clone().detach())
        self.U_history.append(self.U.clone().detach())
        self.v_t_history.append(self.v_t.clone().detach())
        self.z_t_history.append(self.z_t.clone().detach())   


class Dale_CB_STP_batch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(Dale_CB_STP_batch, self).__init__()
        self.rnncell = Dale_CB_STPcell(input_size, hidden_size, num_layers).to(device)
        self.batch_first = batch_first

    def forward(self, x):
        if self.batch_first == True:
            for n in range(x.size(1)):
                #print(x.shape)
                x_slice = torch.transpose(x[:,n,:], 0, 1)
                self.rnncell(x_slice)
        return self.rnncell.excitatory            
            
class Dale_CB_STP(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Dale_CB_STP, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = Dale_CB_STP_batch(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, 10)
        pass

    def forward(self, x):
        # Set initial hidden and cell states 
        self.lstm.rnncell.X = torch.ones(self.hidden_size, x.size(0), dtype=torch.float32).to(device)
        self.lstm.rnncell.U = (self.lstm.rnncell.Ucapclone.repeat(1, x.size(0))).to(device)
        self.lstm.rnncell.v_t = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # Passing in the input and hidden state into the model and  obtaining outputs
        out = self.lstm(x)  # out: tensor of shape (batch_size, hidden_size)
        output_mask = torch.ones_like(out)
        output_mask[:,:self.hidden_size//4] = 0
        output_mask[:,3*self.hidden_size//4:] = 0        
        out = out * output_mask

        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out)
        return out.squeeze(-1)
        
        pass                                    
pass

model = Dale_CB_STP(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
'Training'
print(model)
loss_func = nn.CrossEntropyLoss()

'Trajactory Tracking and Training'
from torch import optim
model_optimizer = optim.Adam(model.parameters(), lr = learning_rate)   

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit

def subset_loader(full_dataset, batch_size, subset_ratio=0.1):
    # Generate labels array to use in stratified split
    labels = []
    for _, label in full_dataset:
        labels.append(label)
    labels = np.array(labels)

    # Perform a stratified shuffle split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=subset_ratio, random_state=0)
    for train_index, test_index in sss.split(np.zeros(len(labels)), labels):
        stratified_subset_indices = test_index

    # Create a Subset instance with the stratified subset indices
    stratified_subset = Subset(full_dataset, stratified_subset_indices)

    # Create DataLoader from the subset
    subset_loader = DataLoader(
        stratified_subset,
        batch_size=batch_size,
        shuffle=False  # No need to shuffle as we already have a random subset
    )

    return subset_loader
subtest = subset_loader(test_data, batch_size)

def evaluate_while_training(model, loaders):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in subtest:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

def train(num_epochs, model, loaders, patience=2, min_delta=0.01):
    model.train()
    total_step = len(loaders['train'])
    train_acc = []
    best_acc = 0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            images = stride(images, stride_number).to(device)
            labels = labels.to(device)
            model.train()
            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Backward and optimize
            model_optimizer.zero_grad()
            loss.backward()
            model_optimizer.step()
            
            if (i+1) % 750 == 0:
                accuracy = evaluate_while_training(model, loaders)
                train_acc.append(accuracy)
                print('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.2f}' 
                      .format(epoch + 1, num_epochs, i + 1, total_step, accuracy))

                # Check for improvement
                if accuracy - best_acc > min_delta:
                    best_acc = accuracy
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1

                if no_improve_epochs >= patience:
                    print("No improvement in validation accuracy for {} epochs. Stopping training.".format(patience))
                    return train_acc

    return train_acc

train_acc = train(num_epochs, model, loaders)


In [ ]:
'Testing Accuracy'
# Test the model
model.eval()
X_history = []
U_history = []
v_t_history = []
z_t_history = []
labelslist = []

with torch.no_grad():
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        model.lstm.rnncell.X_history = []
        model.lstm.rnncell.U_history = []
        model.lstm.rnncell.v_t_history = []
        model.lstm.rnncell.z_t_history = []
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = stride(images, stride_number).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()
        X_history.append(model.lstm.rnncell.X_history)
        U_history.append(model.lstm.rnncell.U_history)
        v_t_history.append(model.lstm.rnncell.v_t_history)
        z_t_history.append(model.lstm.rnncell.z_t_history)
        labelslist.append(labels)

test_acc = 100 * correct / total
print('Accuracy of the model:{}%'.format(test_acc))

In [ ]:
# Save Model
P = model.lstm.rnncell.P.detach().cpu().numpy()
W = model.lstm.rnncell.W.detach().cpu().numpy()
read_out = model.fc.weight.detach().cpu().numpy()
K = model.lstm.rnncell.K.detach().cpu().numpy()
C = model.lstm.rnncell.C.detach().cpu().numpy()
P_z = model.lstm.rnncell.P_z.detach().cpu().numpy()
b_z = model.lstm.rnncell.b_z.detach().cpu().numpy()
e_e = model.lstm.rnncell.e_e.detach().cpu().numpy()
e_i = model.lstm.rnncell.e_i.detach().cpu().numpy()
b_v = model.lstm.rnncell.b_v.detach().cpu().numpy()
Ucap = model.lstm.rnncell.Ucap.detach().cpu().numpy()
c_U = model.lstm.rnncell.c_U.detach().cpu().numpy()
c_u = model.lstm.rnncell.c_u.detach().cpu().numpy()
c_x = model.lstm.rnncell.c_x.detach().cpu().numpy()

import pickle
with open('weights24/07_Dale-CB-STP_48.pkl', 'wb') as f:
    pickle.dump([P, W, read_out, K, C, P_z, b_z, e_e, e_i, b_v, Ucap, c_U, c_u, c_x], f)
    pickle.dump([X_history, U_history, v_t_history, z_t_history, labelslist], f)
    pickle.dump([input_size, hidden_size, num_layers, num_classes, batch_size, num_epochs, learning_rate, stride_number], f)
    f.close()


In [ ]:
# save the trained model
torch.save(model.state_dict(), 'weights24/07_Dale-CB-STP_48.pth')